# Quick start -- Use Case Example
For this example is used the [UCI adult dataset](https://archive.ics.uci.edu/ml/datasets/Adult) where the objective is to predict whether a person makes more (label 1) or less (0) than $50,000 a year.

In [1]:
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier

#Import dataset
d = fetch_openml(data_id=1590, as_frame=True)
X = d.data
d_train=pd.get_dummies(X)
y_true = (d.target == '>50K') * 1

#training the classifier
classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(d_train, y_true)

#Producing y_pred
y_pred = classifier.predict(d_train)

## Use of the FairSD package
Here we use the DSSD (Diverse Subgroup Set Discovery) algorithm and the demographic_parity_difference (from Fairlearn) to find the top-k (k = 5 by default) discriminated subgroups.<br/>
The execute method return a **ResultSet object**.

In [2]:
import fairsd as fsd
task=fsd.SubgroupDiscoveryTask(X, y_true, y_pred, qf = "demographic_parity_difference")
result_set=fsd.DSSD().execute(task)

### ResultSet object

We can transform the result set into a dataframe as shown below. Each row of this dataframe represents a subgroup.

In [3]:
df=result_set.to_dataframe()
display(df)

,quality,description,size,proportion
sg0,0.913502,education = 'Bachelors' AND marital-status = '...,4136,0.084681
sg1,0.913502,"education-num = '(12.0, 13.0]' AND marital-sta...",4136,0.084681
sg2,0.866879,"capital-gain = '(6849.0, +infinite]' AND fnlwg...",2036,0.041685
sg3,0.863130,education = 'Masters' AND marital-status = 'Ma...,1527,0.031264
sg4,0.853604,"education-num = '(14.0, +infinite]' AND marita...",999,0.020454


We can also print the result set or convert it into a string as shown below.

In [4]:
resultset_string = result_set.to_string()
print(result_set)

education = 'Bachelors' AND marital-status = 'Married-civ-spouse' 
education-num = '(12.0, 13.0]' AND marital-status = 'Married-civ-spouse' 
capital-gain = '(6849.0, +infinite]' AND fnlwgt = '(24763.0, +infinite]' 
education = 'Masters' AND marital-status = 'Married-civ-spouse' 
education-num = '(14.0, +infinite]' AND marital-status = 'Married-civ-spouse' 



### Generate a feature from a subgroup
ResultSet basically contains a list of subgroup descriptions ([Description](https://github.com/MaurizioPulizzi/fairsd/blob/main/fairsd/sgdescription.py#L80) object).<br/>
Another intresting method of Resultset object allow us to 
**select a subgroup X from the result set and automatically generate the feature "Belong to subgroup X"**.This is very useful for deepening the analysis on the found subgroups, for example we can use the FairLearn library for this purpose.<br/>
An example is shown below:

In [12]:
from fairlearn.metrics import MetricFrame
from fairlearn.metrics import selection_rate

# Here we generate the feature "Belong to subgroup n. 0"
# The result is a pandas Series. The name of this Series is "sg0".
# This series contains an element for each instance of the dataset. Each element is True 
# iff the istance belong to the subgroup sg0
sg_feature = result_set.sg_feature(sg_index=0, X=X)

# Here we basically use the FairLearn library to further analyzing the subgroup sg0
selection_rate = MetricFrame(selection_rate, y_true, y_pred, sensitive_features=sg_feature)
print(selection_rate.by_group)

sg0
False    0.0864985
True             1
Name: selection_rate, dtype: object
